In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import torch as torch
import torch.nn.functional as F
import rasterio as rio
import rasterio.windows

import copy

from tqdm.notebook import tqdm

from hyperspectral.math import zca_whitening_matrix
from hyperspectral.band_selection import *

# Explore #

## Clip Spectrum ##

In [ ]:
filename = 'data2/spectra/0/splib07a_Red_Coated_Algea_Water_RCAW1_ASDFRa_AREF.txt'
spectrum = []
with open(filename, 'r') as f:
    f.readline()
    for line in f.readlines():
        spectrum.append(float(line))
spectrum = np.array(spectrum)
spectrum = spectrum / np.linalg.norm(spectrum[0:993], ord=2) # only the first 993 bands are good
spectrum[993:] = 0
spectrum_normalized = scipy.signal.resample(spectrum, 224) - spectrum.mean()
spectrum_normalized -= np.min(spectrum_normalized[:103])
spectrum_normalized[103:] = 0

In [ ]:
plt.plot(spectrum_normalized)

In [ ]:
spectrum.shape

## Perform Inference ##

In [ ]:
tile_size = 64
def infer1(in_filename, out_filename):
    with rio.open(in_filename, 'r') as in_ds:
        profile = copy.deepcopy(in_ds.profile)
        profile.update(count=1, driver='GTiff', bigtiff='yes', compress='deflate', predictor='2', tiled='yes', dtype=np.float32, sparse_ok='yes')
        with rio.open(out_filename, 'w', **profile) as out_ds:
            for col in tqdm(range(0, in_ds.width, tile_size), position=0):
                width = min(col+tile_size, in_ds.width) - col
                for row in tqdm(range(0, in_ds.height, tile_size), position=1, leave=False):
                    height = min(row+tile_size, in_ds.height) - row
                    window = rasterio.windows.Window(col, row, width, height)
                    data = in_ds.read(1, window=window)
                    if np.abs(data).sum() == 0:
                        continue
                    data = np.transpose(in_ds.read(window=window).astype(np.float32), (1,2,0))
                    norm = np.linalg.norm(data[:,:,103:], ord=2, axis=2)[..., None].astype(np.float32)
                    data /= norm
                    data -= np.mean(data, axis=2)[...,None]
                    data = np.dot(data, spectrum_normalized)
                    data[np.isnan(data)] = 0
                    data = data.reshape(1, width, height).astype(np.float32)
                    out_ds.write(data, window=window)

In [ ]:
in_out = [
    ('data2/LakeErie/f080723t01p00r06/f080723t01p00r06rdn_c_sc01.tif', 'data2/f080723t01p00r06rdn_c_sc01_result.tif'),
    ('data2/LakeMichigan/f080709t01p00r13/f080709t01p00r13rdn_c_sc01.tif', 'data2/f080709t01p00r13rdn_c_sc01_result.tif')
]

In [ ]:
for (in_filename, out_filename) in in_out:
    infer1(in_filename, out_filename)

# Save Samples #

In [ ]:
in_filename1 = 'data2/LakeMichigan/f080709t01p00r13/f080709t01p00r13rdn_c_sc01.tif'
in_filename2 = 'data2/f080709t01p00r13rdn_c_sc01_result.tif'

In [ ]:
tile_size = 64
algea_yes = []
algea_no = []
with rio.open(in_filename1, 'r') as ds1, rio.open(in_filename2, 'r') as ds2:
    width, height = ds1.width, ds1.height
    for col in tqdm(range(0, width, tile_size), position=0):
        for row in tqdm(range(0, height, tile_size), position=1, leave=False):
            window = rasterio.windows.Window(col, row, tile_size, tile_size)
            data0 = ds1.read(1, window=window).reshape(-1,1)
            if np.abs(data0).sum() == 0:
                continue
            data1 = np.transpose(ds1.read(window=window).astype(np.float32), (1,2,0)).reshape(-1,224)
            data2 = np.transpose(ds2.read(window=window).astype(np.float32), (1,2,0)).reshape(-1,1)
            algea_yes.append(data1[np.squeeze(data2 > +1.8)])
            algea_no.append(data1[np.squeeze((data2 < +1.0) * (data0 != 0))])

In [ ]:
algea_yes = np.concatenate(algea_yes)
algea_no = np.concatenate(algea_no)

In [ ]:
np.savez('data2/algea.npz', algea_yes=algea_yes, algea_no=algea_no, spectrum=spectrum)

# Band Selection and Whitening #

## Whitening ##

In [ ]:
dictionary = np.load('data2/algea.npz')
pos = dictionary.get('algea_yes')
neg = dictionary.get('algea_no')
spectrum = dictionary.get('spectrum')

spectrum = scipy.signal.resample(spectrum, 224) - spectrum.mean()

# Only the first 103 bands are usable
spectrum = spectrum[:103]
pos = pos[:,:103]
neg = neg[:,:103]

pos /= np.linalg.norm(pos, ord=2, axis=1).reshape(-1,1)
neg /= np.linalg.norm(neg, ord=2, axis=1).reshape(-1,1)

pos -= np.mean(pos, axis=1).reshape(-1,1)
neg -= np.mean(neg, axis=1).reshape(-1,1)

In [ ]:
W, mean = zca_whitening_matrix(neg)

In [ ]:
def whiten(m, W, mean):
    old_shape = m.shape
    m = m.reshape(-1, old_shape[-1])
    m = m - mean
    m = np.matmul(m, W)
    m = m.reshape(*old_shape)
    return m

In [ ]:
whitened_pos = whiten(pos, W, 0)
whitened_neg = whiten(neg, W, 0)
whitened_spectrum = whiten(spectrum, W, 0)

In [ ]:
np.dot(whitened_pos, whitened_spectrum).mean()

In [ ]:
np.dot(whitened_neg, whitened_spectrum).mean()

In [ ]:
channels = tuple(range(0+1,103+1))
tile_size = 64
def infer2(in_filename, out_filename):
    with rio.open(in_filename, 'r') as in_ds:
        profile = copy.deepcopy(in_ds.profile)
        profile.update(count=1, driver='GTiff', bigtiff='yes', compress='deflate', predictor='2', tiled='yes', dtype=np.float32, sparse_ok='yes')
        with rio.open(out_filename, 'w', **profile) as out_ds:
            for col in tqdm(range(0, in_ds.width, tile_size), position=0):
                width = min(col+tile_size, in_ds.width) - col
                for row in tqdm(range(0, in_ds.height, tile_size), position=1, leave=False):
                    height = min(row+tile_size, in_ds.height) - row
                    window = rasterio.windows.Window(col, row, width, height)
                    data = in_ds.read(1, window=window)
                    if np.abs(data).sum() == 0:
                        continue
                    data = np.transpose(in_ds.read(channels, window=window).astype(np.float32), (1,2,0))
                    norm = np.linalg.norm(data, ord=2, axis=2)[..., None].astype(np.float32)
                    data /= norm
                    data -= np.mean(data, axis=2)[...,None]
                    data = whiten(data, W, 0)
                    data = np.dot(data, whitened_spectrum)
                    data[np.isnan(data)] = 0
                    data = data.reshape(1, width, height).astype(np.float32)
                    out_ds.write(data, window=window)

In [ ]:
out_filename = 'data2/f080709t01p00r13rdn_c_sc01_result_whitened.tif'
infer2(in_filename1, out_filename)